<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/TestSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: MapReduce & Web Graph

## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class :😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Staff will allow your notebook to **execute from start to finish for no more than 90 seconds**, then manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell. 

**Submission**: Upload a zip file **with your ID as the file name** containing the colab notebook and the GCP notebook to the assignment on Moodle. The names of the files should be  in the format:  012345678_colab.ipynb and 012345678_gcp.ipynb.

# Tasks

In this assignment, we are going to use MapReduce in pySpark to scale up index creation. **The assignment consists of two parts, one in Google Colab and one in GCP**. In the first part (Colab), we will learn how to use MapReduce in PySpark, create an inverted index, and calculate PageRank. In the second part, we will use GCP to scale up index creation for the entire English Wikipedia. 

You will learn how to work with PySpark, write MapReduce code that executes on a cluster of machines, and experience the power of parallel processing. **All tasks must be completed through the use of PySpark's MapReduce**.

To save you (and us) time, both in this assignment and in the final project, we are providing you with preprocessed files for the entire English wikipedia dump. Preprocessing was done, similar to Assignment #2, to extract article information from the XML, then remove the WikiMedia markup. The complete preprocessing code for one wiki dump can be seen [here](https://colab.research.google.com/drive/1xXc0YPU4BVd8CAjVNgICimW_d-Z7wryu?usp=sharing).


**By the end of this assignment, you will have:**

In Colab (65 Points)

1. (20 Points) Processed articles' body to produce posting lists (using PySpark and MapReduce).
2. (5 Points) Calculated document frequency (DF) in a  distributed fashion. 
3. (20 Points) Partitioned the posting lists and wrote them to disk in parallel.
4. (20 Points) Constructed the links graph and calculated PageRank on it. 
All of these will be based on a small subset of articles from one wiki dump file.

In GCP (35 Points):

1. (10 Points) Successfully started a Dataproc cluster, setup cloud storage bucket, and properly initialized cluster nodes.
2. (10 Points) Created an inverted index for all of English Wikipedia. 
3. (10 Points) Successfully computed PageRank for the entire English Wikipedia.
4. (5 Points) Reporting: size of the data, size of index, time it took to build the index, time it took to compute PageRank, and the $ amount of credits it consumed.

Bonus (10 points) if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 







# Setup

## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
--2023-01-12 17:50:01--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 108.156.83.116, 108.156.83.37, 108.156.83.15, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|108.156.83.116|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [5]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. 

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the 
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. 
Notice that you have to change the project_id variable in the second cell below. 

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 300000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except: 
   pass

In [7]:
wiki_data = "wikidata20210801_preprocessed"

In [8]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'ass3-370307'
!gcloud config set project {project_id}

data_bucket_name = wiki_data
try:
    if os.environ[wiki_data] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
\ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark. 

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory. 

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [9]:
from pathlib import Path 
import os

try:
    if os.environ[wiki_data] is not None:
      path = os.environ[wiki_data]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd




In [11]:
# doc_text_pairs.map()

In [12]:
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd

In [13]:
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

## Word counts

### Term frequency

**YOUR TASK (10 POINTS):** Complete the implementation of `word_count`, which calculates the term-frequency (tf) for each word in a single document.  map function. The functions gets the text and the wiki id, calculates the tf and returns list of tuples in the form of (token, (wiki_id, tf)).

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)



def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  hash={}

  for word in tokens:
    if word in all_stopwords:
      continue
    if word in hash:
      hash[word] = (id, hash[word][1]+1)
      # term_total_a[word] = term_total_a[word]+1

    else:
      hash[word] = (id,1)
      # term_total_a[word] = 1

  # print(list(hash.items()))
  # print
  return list(hash.items())

In [15]:
# test for word count
text = "This is a test to check the words count function actually count words"
id = 0
word_count_results = word_count(text, id)



assert word_count_results[0][0] == 'test'
assert len(word_count_results) == 6
assert word_count_results[2][1][1] == word_count_results[2][1][1]
assert type(word_count_results[0][0]) == str
assert type(word_count_results[0][1]) == tuple


PySpark has a `map` trasformation that, just like we learned in class about Map in MapReduce, executes a given function on each element of an  RDD. It also has a `flatMap()` transformation that executes a given function on each element of an RDD, but afterwards flattens the results to create one big "list" instead multiple nested lists. For example, `flatMap` on the above `word_count` function will concatenate the tokens from multiple document into one big list rather than keeping separate list of tokens for each document like a regular `map` would produce. 

In [16]:
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))

After `flatMap` is executed, the counts for a given token are likely to be scattered accross multiple machines/workers in the cluster. In order to create the posting lists for each token, we need a reduce operation that collects them into one list and sorts them by the document id (wiki_id). 

Spark has several ways of reducing or aggergating values, but we will use `groupByKey`, which like Reduce in MapReduce, collects all the values for a given key into an iterable list of the values associated with that key. The result of `groupByKey` is an RDD with (key, value*s*) pairs. Then, we can call `map` to execute a function on each pair in that RDD or call `mapValues` to execute a function only on the values iterable.

**YOUR TASK (5 POINTS):** Complete the implementation of `reduce_word_counts`, which operates on the pairs returned by `word_count`. The functions gets a list of values (unsorted posting list) and sorts it.

In [17]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x:x[0])

In [18]:
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

In [19]:
## test reduce_word_count
token, posting_list = postings_text.take(1)[0]
assert type(token) == str
assert type(posting_list) == list
doc_id, tf = zip(*posting_list)
assert type(doc_id[0]) == int
assert type(tf[0]) == int
assert np.diff(doc_id).min() > 0 
assert np.min(tf) > 0 
assert postings_text.count() == 153014

### Document frequency and filteration

Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents). 

In [20]:
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>10)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>10)
postings_filtered_anchor = postings_anchor.filter(lambda x: len(x[1])>10)

Now, we would like to calculate the df for each word.

**YOUR TASK (5 POINTS):** Complete the implementation of `calculate_df` that takes a posting list RDD and returns an RDD where each element is a (token, df) pair. 

In [21]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map( lambda x: (x[0], len(x[1])) )

In [22]:
# global statistics
w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)
w2df_anchor = calculate_df(postings_filtered_anchor)



# test calculate_df types
token, df = w2df_text.take(1)[0]
assert type(token) == str
assert type(df) == int
# test min/max df values
# collectAsMap collects the results to the master node's memory as a dictionary.
# we know it's not so big so this is okay.
w2df_dict = w2df_text.collectAsMap()
assert np.min(list(w2df_dict.values())) == 11
assert np.max(list(w2df_dict.values())) == 819
# test select words
assert w2df_dict['first'] == 805
assert w2df_dict['many'] == 670
assert w2df_dict['used'] == 648

## Partitioning and writing the index

Now that we have the postings information we need to write it to disk. We would like to partition the tokens and posting lists into buckets so when we get a query, we know exactly in which bucket to look for its posting list. This can also help parallelize retrieval from the index if we have multiple machines answering queries. 

Here, you are asked to do this partitioning/bucketing through another round of map and reduce operations, where the last step should be to write out each bucket into a separate inverted index on disk. Use the staff-provided function of  `write_a_posting_list` implemented in inverted_index_colab.py for the InvertedIndex class. Eventually, this partitioning and writing process needs to return information about where each posting list is located (file name and offset) for a token. 

**YOUR TASK (20 POINTS):** Complete the implementation of `partition_postings_and_write` that partitions the posting list, writes out each bucket, and returns information about the location on disk of each posting list.

In [23]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE

  result = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey().map(lambda item: InvertedIndex.write_a_posting_list(item))

  # result = hashItems.map(lambda item: InvertedIndex.write_a_posting_list(item))
  # array_of_items = []

  # for item in hashItems.collect():
  #   posting_locations_dictionary = InvertedIndex.write_a_posting_list(item)
  #   array_of_items.append(posting_locations_dictionary)

  # rdd = sc.parallelize(array_of_items)
  
  return result

The next cell uses the `collect` operator to aggregate the posting location information from the multiple buckets/indices into a single dictionary. We know in this case that the posting locations info is relatively small so this is not going to crash our master node. 

In [25]:
# test partitioning for the different buckets (10 points)
posting_locs_list = partition_postings_and_write(postings_filtered_text).collect()
assert len(posting_locs_list) == 124
# assert len(list(Path('.').glob('*.bin'))) == 124
pl_lens = [len(pl) for pl in posting_locs_list]
assert np.max(pl_lens) == 141
assert np.min(pl_lens) == 91 
assert int(np.median(pl_lens)) == 116

In [26]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

assert super_posting_locs['try'][0][0][:3] == '105'
assert super_posting_locs['political'][0][0][:2] == '83'
assert 'hi' not in super_posting_locs
assert len(super_posting_locs) == 14524

Putting everything together (posting locations, df) and test that the resulting index is correct. 

In [27]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

# inverted.term_total = term_total_a
# write the global stats out
inverted.write_index('.', 'index')

In [28]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [29]:
## test the final index
pl = read_posting_list(inverted, 'clarity')
assert pl[-3] == (2204, 2)
assert len(pl) == 18

In [30]:
TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

nltk.download('stopwords')
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords) #this is containing all the stop words
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in english_stopwords]
    return list_of_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# def posting_lists_reader(inverted_index, term): # work 2
#     """  reads one posting list from disk in byte and convert to int
#         return:
#             [(doc_id:int, tf:int), ...].
#     """
#     with closing(MultiFileReader()) as reader:
#         locs = inverted_index.posting_locs[term]
#         b = reader.read(locs, inverted_index.df[term] * TUPLE_SIZE)# convert the bytes read into `b` to a proper posting list.
#         posting_list = []

#         while b:
#             b_doc_id = int.from_bytes(b[0:4], "big")
#             b_tf_of_w = int.from_bytes(b[4:6], "big")
#             b = b[6:]
#             posting_list.append((b_doc_id, b_tf_of_w))

#         return posting_list


def posting_lists_reader(inverted_index, term):
    """ A generator that reads one posting list from disk and yields
        a (word:str, [(doc_id:int, tf:int), ...]) tuple.
    """
    with closing(MultiFileReader()) as reader:
        locs = inverted_index.posting_locs[term]
        b = reader.read(locs, inverted_index.df[term] * TUPLE_SIZE)
        posting_list = []
        for i in range(inverted_index.df[term]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))

        return posting_list

In [32]:
def get_top_n(lst, N = 5): #sort the list according to the x[1] and return the top N
    return sorted(lst, key= lambda x:x[1], reverse=True)[:N]

In [33]:
# term_total_a = {}

# def counterA(row):
#   tokens = tokenize(row['text'])
#   for token in tokens:
#     if token in term_total_a:
#       term_total_a[token] = term_total_a[token]+1

#     else:
#       term_total_a[token] = 1

    
      

# arr2 = doc_text_pairs.map(counterA)




In [34]:
# arr2

In [35]:
DL = {}




def counter(row):
  tokens = tokenize(row['text'])
  # print(len(tokens))
  # DL[row['id']] = len(tokens)
  return (row['id'],len(tokens))

arr = doc_text_pairs.map(counter)

In [36]:
DL = arr.collectAsMap()
DL

{12: 4224,
 25: 4301,
 39: 1892,
 290: 1051,
 303: 8360,
 305: 4441,
 307: 8224,
 308: 6309,
 309: 1054,
 316: 4637,
 324: 4503,
 330: 262,
 332: 249,
 334: 701,
 336: 3932,
 339: 4368,
 340: 329,
 344: 842,
 358: 6840,
 359: 2623,
 569: 4301,
 572: 644,
 573: 5029,
 579: 448,
 580: 476,
 586: 3481,
 590: 199,
 593: 3744,
 594: 10307,
 595: 5546,
 597: 2352,
 599: 3273,
 600: 5307,
 612: 761,
 615: 923,
 620: 3879,
 621: 7135,
 624: 7342,
 627: 4899,
 628: 2619,
 630: 379,
 632: 640,
 633: 3091,
 634: 2833,
 639: 4118,
 640: 1983,
 642: 149,
 643: 622,
 649: 577,
 651: 1152,
 653: 2969,
 655: 2257,
 656: 3160,
 657: 4681,
 659: 1043,
 661: 136,
 662: 6868,
 663: 4581,
 664: 2896,
 665: 1562,
 666: 9509,
 670: 2842,
 673: 1005,
 674: 3800,
 675: 413,
 676: 3432,
 677: 2190,
 678: 286,
 679: 586,
 680: 1820,
 681: 1082,
 682: 1498,
 683: 425,
 689: 4626,
 690: 3023,
 691: 3856,
 694: 42,
 698: 5137,
 700: 7601,
 701: 6092,
 704: 1134,
 705: 708,
 706: 1853,
 708: 235,
 709: 1305,
 710: 1

In [37]:
import math
def query_get_top_N_tfidf(inverted_index, query_to_search, N = 5):
    """

    Args:
        query_to_search:
        index:

    Returns:

    """
    

    # for embedding in query_to_search:
    #   print(embedding)

    result = {}  # result[doc_id] = score
    epsilon = .0000001
    counter = Counter(query_to_search)
    DL_length = len(DL)
    query_length = len(query_to_search)

    for token in np.unique(query_to_search):
        # print(token)
        # print(inverted_index.df.keys())
        if token in inverted_index.df.keys():  # avoid terms that do not appear in the index.
            tf = counter[token] / query_length  #term frequency divded by the length of the query
            df = inverted_index.df[token]
            idf = math.log((DL_length) / (df + epsilon), 10)  #todo ass4 -> make save DL in memory We save a dictionary named DL, which fetches the document length of each document.
            docs = posting_lists_reader(inverted_index,token) #will return the list of docs, from byte to  [(doc_id:int, tf:int), ...]
            Q = tf*idf

            # print()
            for doc_id, doc_tf in docs:
                D = (doc_tf / DL[doc_id]) * math.log(DL_length / inverted_index.df[token], 10)
                tfidf = D*Q
                result[doc_id] = result.get(doc_id, 0) + tfidf

    return get_top_n(result.items(),N)

In [38]:
#this is word embeding רשת נוירונים 
from gensim.models import Word2Vec
word2vTraining = [['best','air','jordan','hello', 'marvel', 'movie', 'How', 'do', 'kids', 'come', 'to', 'world', 'Information', 'retrieval', 'LinkedIn', 'How', 'to', 'make', 'coffee', 'Ritalin', 'What', 'is', 'the', 'best', 'way', 'to', 'invest', 'money', 'How', 'to', 'lose', 'weight', 'fast', 'What', 'is', 'the', 'best', 'way', 'to', 'improve', 'my', 'memory', 'How', 'to', 'improve', 'my', 'mental', 'health', 'What', 'is', 'the', 'best', 'way', 'to', 'improve', 'my', 'relationships', 'How', 'to', 'improve', 'my', 'communication', 'skills']]
# word2vTraining.append("air")
model = Word2Vec(word2vTraining, min_count=0, alpha=0.01)
model.save("word2vec.bin")
model = Word2Vec.load("/content/word2vec.bin")


In [115]:
from threading import excepthook
print(list(model.wv.vocab))

def search(query):
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    # res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION

    # print(model)

    tokens = tokenize(query.lower())

    # embeddings = []

    # for w in tokens:
    #   p = []
    #   try:
    #       p=list(model.most_similar(w)[:3])
    #   except:
    #     print()
    #   # if len(p) == 0 :
    #   p += [(w,0)]
    #   embeddings += p

    # embeddings = list(map(lambda x:x[0], embeddings))
    # print(embeddings)
    bestDocs = query_get_top_N_tfidf(inverted,tokens,20)
    best = set(map(lambda x:x[0],bestDocs))

    filtered_rdd = doc_title_pairs.filter(lambda x: x['id'] in best)

    # Extract the titles from the filtered RDD
    res = filtered_rdd.map(lambda x: (x['id'],x['title'])).collect()
    # res = bestDocs
    # print(res)
    return res



['best', 'air', 'jordan', 'hello', 'marvel', 'movie', 'How', 'do', 'kids', 'come', 'to', 'world', 'Information', 'retrieval', 'LinkedIn', 'make', 'coffee', 'Ritalin', 'What', 'is', 'the', 'way', 'invest', 'money', 'lose', 'weight', 'fast', 'improve', 'my', 'memory', 'mental', 'health', 'relationships', 'communication', 'skills']


In [116]:
search("apple")

[(308, 'Aristotle'),
 (653, 'Assistive technology'),
 (785, 'Asteroids (video game)'),
 (856, 'Apple Inc.'),
 (887, 'MessagePad'),
 (1344, 'Apple I'),
 (1449, 'Alan Kay'),
 (1788, 'ABM'),
 (1955, 'Adobe Inc.'),
 (2100, 'Applesoft BASIC'),
 (2114, 'IBM AIX'),
 (2115, 'AppleTalk'),
 (2116, 'Apple II series'),
 (2117, 'Apple III'),
 (2137, 'Aster CT-80'),
 (2142, 'List of artificial intelligence projects'),
 (2245, 'Annapolis Valley'),
 (2251, 'Accusative case'),
 (2275, 'Apple II'),
 (2316, 'Audio file format')]

In [112]:
from math import log

length_docLengths = len(DL) #to run faster one time calculate
dl_list =  list(DL.items())
dl_list =list(map(lambda x: int(x[1]),dl_list))
# print(dl_list)
sum_dl = 0
for n in dl_list:
  sum_dl += n

avg_dl = sum_dl / length_docLengths

class BM25:
    def __init__(self, inverted_index,k1,k3,b):
        self.inverted_index = inverted_index
        self.doc_lengths = DL
        self.N = length_docLengths
        self.avgdl = avg_dl
        self.b = b
        self.k1 = k1
        self.k3 = k3

    def get_scores(self, tokens):
            query_Counter = Counter(tokens)
            print(query_Counter)
            scores = {}
            for term in tokens:
                if term not in self.inverted_index.df.keys():
                    continue

                idf = math.log10((self.N+1)/self.inverted_index.df[term])
                posting = posting_lists_reader(self.inverted_index, term)

                for doc_id, tf in posting:
                  d = self.doc_lengths[doc_id]
                  tf_ij = ((self.k1+1)*tf)/((1 - self.b + self.b * d / self.avgdl)*self.k1+tf)
                  tf_iq = ((self.k3+1)*query_Counter[term])/(self.k3+query_Counter[term])
                  score =  tf_ij* idf  * tf_iq
                  if doc_id not in scores:
                      scores[doc_id] = 0
                  scores[doc_id] += score
            return scores


def bm25(inverted_index,tokens,k1,k3,b,N):
  b = b 
  bm = BM25(inverted_index,k1,k3,b)
  result = bm.get_scores(tokens)
  return get_top_n(list(result.items()),N),

  



    


In [113]:
# bm25("house")

query = "apple company"
tokens = tokenize(query)
k1 = 0.5
k2 = 0.5
b = 0.5
bestDocs = bm25(inverted,tokens,k1,k2,b,10)
print(bestDocs)
best = set(map(lambda x:x[0],bestDocs[0]))
print(best)

Counter({'apple': 1, 'company': 1})
([(856, 2.730262304515795), (2116, 2.6795735469221853), (1344, 2.675220626505635), (2117, 2.654053765509328), (2275, 2.6524619330469896), (1955, 2.601393256557506), (887, 2.5480722878793403), (1980, 2.488381813683913), (2100, 2.473726938748926), (2137, 2.46702713528526)],)
{1344, 2275, 2116, 2117, 1955, 2100, 887, 856, 2137, 1980}


## PageRank

In this section, your task is to compute PageRank for wiki articles using the anchor text we extracted from the MediaWiki markdown. Let's select the 'id' and 'anchor_text' fields into a new RDD: